In [1]:
import anndata as ad
import duckdb
# import ehrapy as ep
import ehrdata
import numpy as np

In [2]:
# Check for mimic_iv_omop

In [2]:
con = duckdb.connect()
ehrdata.dt.mimic_iv_omop(backend_handle=con)
obs = ehrdata.io.omop.extract_person(con)
obs.head()

Download successful. ZIP file downloaded and extracted successfully to ehrapy_data/mimic-iv-demo-data-in-the-omop-common-data-model-0.9.
missing tables:  [['concept'], ['vocabulary'], ['domain'], ['concept_class'], ['concept_relationship'], ['relationship'], ['concept_synonym'], ['concept_ancestor'], ['source_to_concept_map'], ['drug_strength']]


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,3589912774911670296,8507,2095,None,None,None,0,38003563,None,None,None,10009628,M,0,None,0,HISPANIC/LATINO,2000001408
1,-3210373572193940939,8507,2079,None,None,None,0,38003563,None,None,None,10011398,M,0,None,0,HISPANIC/LATINO,2000001408
2,-775517641933593374,8507,2149,None,None,None,8516,0,None,None,None,10004235,M,0,BLACK/AFRICAN AMERICAN,2000001406,None,0
3,-2575767131279873665,8507,2050,None,None,None,8516,0,None,None,None,10024043,M,0,BLACK/AFRICAN AMERICAN,2000001406,None,0
4,-8970844422700220177,8507,2114,None,None,None,8527,0,None,None,None,10038933,M,0,WHITE,2000001404,None,0


In [ ]:
# Check for gibleed

In [3]:
con_gibleed = duckdb.connect()

In [4]:
ehrdata.dt.gibleed_omop(backend_handle=con_gibleed)

Download successful. ZIP file downloaded and extracted successfully to ehrapy_data/GIBleed_dataset.
ehrapy_data/GIBleed_dataset/GiBleed_5.3
missing tables:  [['cohort_definition']]


In [5]:
obs_gibleed = ehrdata.io.omop.extract_person(con_gibleed)

In [6]:
obs_gibleed.head()

,PERSON_ID,GENDER_CONCEPT_ID,YEAR_OF_BIRTH,MONTH_OF_BIRTH,DAY_OF_BIRTH,BIRTH_DATETIME,RACE_CONCEPT_ID,ETHNICITY_CONCEPT_ID,LOCATION_ID,PROVIDER_ID,CARE_SITE_ID,PERSON_SOURCE_VALUE,GENDER_SOURCE_VALUE,GENDER_SOURCE_CONCEPT_ID,RACE_SOURCE_VALUE,RACE_SOURCE_CONCEPT_ID,ETHNICITY_SOURCE_VALUE,ETHNICITY_SOURCE_CONCEPT_ID
0,6,8532,1963,12,31,1963-12-31,8516,0,None,None,None,001f4a87-70d0-435c-a4b9-1425f6928d33,F,0,black,0,west_indian,0
1,123,8507,1950,4,12,1950-04-12,8527,0,None,None,None,052d9254-80e8-428f-b8b6-69518b0ef3f3,M,0,white,0,italian,0
2,129,8507,1974,10,7,1974-10-07,8527,0,None,None,None,054d32d5-904f-4df4-846b-8c08d165b4e9,M,0,white,0,polish,0
3,16,8532,1971,10,13,1971-10-13,8527,0,None,None,None,00444703-f2c9-45c9-a247-f6317a43a929,F,0,white,0,american,0
4,65,8532,1967,3,31,1967-03-31,8516,0,None,None,None,02a3dad9-f9d5-42fb-8074-c16d45b4f5c8,F,0,black,0,dominican,0


In [7]:
# Check for Synthea27Nj

In [8]:
con_syn = duckdb.connect()

In [9]:
ehrdata.dt.synthea27nj_omop(backend_handle=con_syn)

Download successful. ZIP file downloaded and extracted successfully to ehrapy_data/Synthea27Nj/synthea27nj_omop_csv.
missing tables:  []


In [10]:
obs_syn = ehrdata.io.omop.extract_person(con_syn)

In [11]:
obs_syn.head()

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,1,8507,1998,4,9,1998-04-09,8527,38003564,None,None,None,1007c05b-8d20-8fe6-6790-44622f8316df,M,0,white,0,nonhispanic,0
1,2,8532,2014,10,22,2014-10-22,8527,38003564,None,None,None,13025201-4834-d1ca-c3ca-38d6614438f1,F,0,white,0,nonhispanic,0
2,3,8507,2011,6,10,2011-06-10,8527,38003564,None,None,None,135deb0b-6380-7ef2-41a2-9ecd2f3f90f8,M,0,white,0,nonhispanic,0
3,4,8507,2000,8,14,2000-08-14,8527,38003564,None,None,None,1aeac35f-e2c0-5546-b7f5-ebee8a61b19c,M,0,white,0,nonhispanic,0
4,5,8507,1988,9,22,1988-09-22,8527,38003563,None,None,None,1b196f94-9358-acec-e497-56a8747585f7,M,0,white,0,hispanic,0
